## main ipynb file

In [ ]:
# ! git clone https://github.com/jinmang2/RetroReader.git

Cloning into 'RetroReader'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 155 (delta 51), reused 49 (delta 25), pack-reused 66
Receiving objects: 100% (155/155), 1.18 MiB | 2.39 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/캡스톤_인공지능_jinmang_initial/

/content/drive/MyDrive/Colab Notebooks/캡스톤_인공지능_jinmang_initial


In [ ]:
%cd RetroReader

/content/drive/MyDrive/Colab Notebooks/캡스톤_인공지능_jinmang_initial/RetroReader


In [ ]:
!ls

app.py	 LICENSE	outputs_vanilla  retro_st_demo.gif    wandb
configs  outputs	README.md	 train_GovQuAD_v2.py
data	 outputs_prep1	retro_reader	 train_squad_v2.py


In [ ]:
!pip install datasets

In [ ]:
!pip install transformers
!pip install tqdm

In [ ]:
!pip install wandb

In [ ]:
! python train_GovQuAD_v2.py --configs 'configs/train_ko_electra_small.yaml'

### 전처리 2차

In [ ]:
train_df = pd.read_csv("data/df_train_10_pre11.csv")
dev_df = pd.read_csv("data/df_dev_10_pre11.csv")

- [ ]  이후에 전처리 방향 결정해서, 30 epoch 학습
- [ ]  결과보고, 전처리1 or 2 결정해서 가원님께 전달, dataset 늘려서 학습.
- [ ]  깃허브 올리기
- [ ]  30 epoch 모델 전달.

1) 특수기호 추출하기

In [ ]:
full_df = pd.concat([train_df, dev_df])

In [ ]:
tmp_train = train_df['context'][:30]

In [ ]:
tmp_train[0]

'노후 어린이집 누수 제로 추진 결과 보고\nⅠ 추진 근거\n「영유아보육법 시행령」제12조\n「서울특별시 강남구 영유아 보육 및 지원에 관한 조례」제15조, 제27조\n2021년도 구립어린이집 확충 및 시설관리 계획 (2021.1.12.보육지원과-729호) \nⅡ 추진 개요\n사업명 : 노후 어린이집 누수 긴급 개보수 공사\n사업기간 : 2021. 4. 7. ~ 6. 25.\n총사업비 : 금55,570,000원\n사업대상 : 신사어린이집 외 3개소'

In [ ]:
# 첫 단어 첫번째 문자 확인 후 symbols 에 append
# 추가) 결과 관찰 도중 ] } 처럼 닫는 특수기호들이 추출이 안되는 경우 발생
#     ) 마지막 단어 확인 후 append도 추가함.

import re
symbols = []
pattern = re.compile('[^A-Za-z0-9가-힣]') 
for idx, context in enumerate(full_df['context']):
  # if idx == 5:
  #   break
  for line in context.split("\n"):
    if len(line.split()) > 0:
      first = line.split()[0]
      last = line.split()[-1]
      f = pattern.findall(first)
      l = pattern.findall(last)
      if bool(f): # 첫번째 단어의 특수문자
        symbols += f 
      if bool(l): # 마지막 단어의 특수문자
        symbols += l

In [ ]:
from collections import Counter

result = Counter(symbols)

print(result.most_common())

[('.', 194041), ('*', 122407), ('○', 92279), ('-', 87387), ('□', 79753), (')', 64805), ('ㅇ', 41003), (',', 26462), ('\uf000', 20223), ('❍', 18844), ('※', 8805), ('◦', 7574), ('·', 5327), ('「', 4759), ('(', 3564), ('Ⅱ', 3432), ('Ⅰ', 3339), ('>', 3273), ('<', 3186), ('․', 3007), ('▢', 2719), ('Ⅲ', 2687), ('!', 2558), ('②', 2556), ('」', 2541), (':', 2541), ('①', 2353), ('”', 2252), ('?', 2006), ('“', 1996), ('‘', 1883), (']', 1810), ('③', 1776), ('Ⅳ', 1574), ('%', 1548), ('/', 1449), ('’', 1206), ('】', 1192), ('【', 1170), ('▪', 1111), ('[', 1060), ('~', 941), ('『', 940), ('‣', 892), ('④', 862), ('•', 819), ('‧', 762), ('◇', 729), ('■', 688), ('⇒', 645), ('▸', 604), ('』', 593), ('◈', 566), ('Ⅴ', 558), ('●', 497), ('－', 492), ('∙', 488), ('▶', 485), ('❚', 484), ('｢', 484), ('☞', 456), ('⑤', 432), ('→', 415), ('ㆍ', 391), ('❏', 354), ('◯', 312), ('㎡', 277), ('〇', 265), ('△', 265), ('《', 252), ('=', 249), ('･', 248), ('＊', 242), ('⇨', 241), ('▲', 236), ('&', 231), ('❑', 229), ('》', 228), ('㈜',

In [ ]:
result = dict(result.most_common()).keys()
result

dict_keys(['.', '*', '○', '-', '□', ')', 'ㅇ', ',', '\uf000', '❍', '※', '◦', '·', '「', '(', 'Ⅱ', 'Ⅰ', '>', '<', '․', '▢', 'Ⅲ', '!', '②', '」', ':', '①', '”', '?', '“', '‘', ']', '③', 'Ⅳ', '%', '/', '’', '】', '【', '▪', '[', '~', '『', '‣', '④', '•', '‧', '◇', '■', '⇒', '▸', '』', '◈', 'Ⅴ', '●', '－', '∙', '▶', '❚', '｢', '☞', '⑤', '→', 'ㆍ', '❏', '◯', '㎡', '〇', '△', '《', '=', '･', '＊', '⇨', '▲', '&', '❑', '》', '㈜', 'Ⅵ', '☐', '⑥', '➊', "'", '❖', '➋', '\xad', '◆', '∘', '+', '｣', '¦', '▣', '▷', '〕', '"', '⑦', '➌', '～', '❶', '❷', '×', '〔', '〈', '◉', '∼', '・', '〉', '↑', '◊', '⦁', '√', '⑧', '≪', '≫', '❸', '▫', 'ᆞ', '➁', '➀', '–', '⋅', '➡', '\uf06c', '_', '▹', '‑', '➂', '◌', '⁃', 'Ⅶ', '➍', '￭', '❹', '►', '§', '▴', '❰', '❱', '➠', '—', '￮', '⃞', '⑨', '◾', 'ｏ', '★', '◻', '㎥', '❒', '\uf0a1', '↓', '❐', '֠', '㎞', '੦', '➃', '▌', '#', 'ⅱ', 'ㆁ', '▮', 'ⅰ', '➤', '✔', '：', 'Ⅷ', '◎', '✤', '…', '‐', '\U000f02b1', '⑩', 'ⅲ', '〚', '➎', '\uf06e', '☎', '▆', '⑴', '¡', '➄', 'о', '⧭', '⑵', '\x83', '\U000f02b2', '✓', '$', 

2) 특수기호 제거

2-1) answer에서 제외할 특수기호 추출

In [ ]:
full_df['answers'].head()

0    {'text': ['금55,570,000원'], 'answer_start': [212]}
1    {'text': ['금55,570,000원'], 'answer_start': [212]}
2             {'text': ['11대'], 'answer_start': [189]}
3             {'text': ['11대'], 'answer_start': [189]}
4              {'text': ['4대'], 'answer_start': [155]}
Name: answers, dtype: object

In [ ]:
import re
answer_symbols = []
pattern = re.compile('[^A-Za-z0-9가-힣\s]') 
for idx, ans in enumerate(full_df['answers']):
  ans = eval(ans)['text']
  # if idx == 1000:
  #   break
  if len(ans) == 0:
    continue
  check = pattern.findall(ans[0])
  if bool(check):
    answer_symbols += check
    # print(ans) # 생각보다 answers에 특수기호가 정말 많다는 걸 알게됨...

In [ ]:
answer_result = Counter(answer_symbols)

print(answer_result.most_common())
print("\n\n")
answer_symbol_list = list(dict(answer_result.most_common()).keys())
print(answer_symbol_list)

[(',', 13461), ('.', 6560), (')', 6482), ('(', 6456), ('·', 2329), ('‘', 1573), ('-', 1556), ('’', 1547), ('․', 1271), ('*', 1021), (':', 816), ('%', 777), ('/', 662), ('→', 611), ('~', 535), ('○', 531), ('」', 477), ('「', 436), ('“', 365), ('”', 356), ('&', 308), ('ㅇ', 285), ('ㆍ', 195), ('②', 152), ('‧', 147), ('①', 140), ('③', 127), ("'", 113), ('❍', 111), ('～', 105), ('∼', 92), ('⇒', 85), ('△', 77), ('+', 75), ('▲', 63), ('>', 62), ('』', 56), ('④', 55), ('『', 55), ('｢', 55), ('｣', 55), ('◦', 49), ('㎡', 48), ('･', 47), ('㈜', 46), ('\uf000', 43), ('・', 39), ('➡', 30), ('[', 30), (']', 30), ('※', 26), ('×', 25), ('□', 25), ('•', 24), ('"', 22), ('⇨', 20), ('⑤', 18), ('▴', 18), ('▵', 18), ('!', 16), ('☎', 16), ('<', 15), ('∙', 13), ('◯', 12), ('➀', 11), ('➁', 11), ('➂', 11), ('➋', 11), ('㎞', 10), ('↔', 10), ('=', 9), ('℃', 9), ('➌', 9), ('–', 8), ('‣', 8), ('■', 8), ('▶', 8), ('⋅', 8), ('☞', 8), (';', 8), ('➊', 8), ('⑥', 7), ('〜', 7), ('➍', 7), ('❶', 7), ('❷', 7), ('❸', 7), ('❹', 7), ('$

In [ ]:
answer = ""
for a in answer_symbol_list:
  answer += a

In [ ]:
import string
answer.replace(string.punctuation, '')

'·‘’․→○」「“”ㅇㆍ②‧①③❍～∼⇒△▲』④『｢｣◦㎡･㈜\uf000・➡※×□•⇨⑤▴▵☎∙◯➀➁➂➋㎞↔℃➌–‣■▶⋅☞➊⑥〜➍❶❷❸❹ᆞ〉－▸❏Ⅱ°㎝【】▢\xad☏⌟♣▷ㅋㄷ〔〕❺⟶·金㉮㉯㉰∘識²➃➄➅⌜\U000f02b3↓＊＆ᐧ◈§☆▪年↑➎₂ê±㎘㎥㏊《》ⅰⅱⅲ⁕ⓑⓒ兩₄ｓ₆'

확인해본 결과)
- 반려인ㆍ반려견
- ~ 
이 많이 나오는데 같은 의미로 사용되는 것들이 다양함.  
answer쪽 특수문자 분석 결과 단위가 되는 것 위주로 제외하기로 함.

In [ ]:
answer_except = "±㎘㎥㏊～∼㎡㈜㎞℃〜°㎝金年êｓ"

2-2) context에서 제외할 특수기호 추출

In [ ]:
symbol_list = list(result)

In [ ]:
symbol = ""
for s in symbol_list:
  symbol += s 
print(symbol)

.*○-□)ㅇ,❍※◦·「(ⅡⅠ><․▢Ⅲ!②」:①”?“‘]③Ⅳ%/’】【▪[~『‣④•‧◇■⇒▸』◈Ⅴ●－∙▶❚｢☞⑤→ㆍ❏◯㎡〇△《=･＊⇨▲&❑》㈜Ⅵ☐⑥➊'❖➋­◆∘+｣¦▣▷〕"⑦➌～❶❷×〔〈◉∼・〉↑◊⦁√⑧≪≫❸▫ᆞ➁➀–⋅➡_▹‑➂◌⁃Ⅶ➍￭❹►§▴❰❱➠—￮⃞⑨◾ｏ★◻㎥❒↓❐֠㎞੦➃▌#ⅱㆁ▮ⅰ➤✔：Ⅷ◎✤…‐󰊱⑩ⅲ〚➎☎▆⑴¡➄о⧭⑵󰊲✓$−@⧠➅㎢✍❺➭㎒♠㎎╺―‌１Ⅸ！⑪℃󰊳⌟±┗⁍➯↳➏⌜◀ʻ㎜ⅳ;Ÿ}⑶ᐅᄋ〛◐♣＃⋄󰋮➲➨✜▍㉰➫∎\㎝➐″⁎＜＞✽⧈∐➑㎍⚬{ⓛ┖ü⃝‥|㉮㉯㉱㉲㉳└‰〜⁜⑷㎖󰊴ⅵⅶ➒➓▯㎏⑫➛［］﹡➜㉠㉡⟶Œ）′​➆◤◢┎➝２㉴㉵➪⇑ㅣ❳￦➞⨯✲٠⁕㈔，┞⚫²ⅴ㏊μ㎓㎧₂➟⓫´❋㎠✳⑬⏐❻ＸⅩ⟹ⓐⓑ⧬＋❙．Ž├÷█☏�▼·㉢㎘󰊹⇢∆（


In [ ]:
import string
symbol = symbol.replace('-―‌∙“”·‘’․~‑@%/#{}:;*.∼~～=&×㎢㎒㎎㎜㎍‰㎏㎓㎧㈔μ㏊' + string.punctuation + answer_except, '')

In [ ]:
# 제외할 특수문자
print(symbol)

.*○-□)ㅇ,❍※◦·「(ⅡⅠ><․▢Ⅲ!②」:①”?“‘]③Ⅳ%/’】【▪[~『‣④•‧◇■⇒▸』◈Ⅴ●－∙▶❚｢☞⑤→ㆍ❏◯㎡〇△《=･＊⇨▲&❑》㈜Ⅵ☐⑥➊'❖➋­◆∘+｣¦▣▷〕"⑦➌～❶❷×〔〈◉∼・〉↑◊⦁√⑧≪≫❸▫ᆞ➁➀–⋅➡_▹‑➂◌⁃Ⅶ➍￭❹►§▴❰❱➠—￮⃞⑨◾ｏ★◻㎥❒↓❐֠㎞੦➃▌#ⅱㆁ▮ⅰ➤✔：Ⅷ◎✤…‐󰊱⑩ⅲ〚➎☎▆⑴¡➄о⧭⑵󰊲✓$−@⧠➅㎢✍❺➭㎒♠㎎╺―‌１Ⅸ！⑪℃󰊳⌟±┗⁍➯↳➏⌜◀ʻ㎜ⅳ;Ÿ}⑶ᐅᄋ〛◐♣＃⋄󰋮➲➨✜▍㉰➫∎\㎝➐″⁎＜＞✽⧈∐➑㎍⚬{ⓛ┖ü⃝‥|㉮㉯㉱㉲㉳└‰〜⁜⑷㎖󰊴ⅵⅶ➒➓▯㎏⑫➛［］﹡➜㉠㉡⟶Œ）′​➆◤◢┎➝２㉴㉵➪⇑ㅣ❳￦➞⨯✲٠⁕㈔，┞⚫²ⅴ㏊μ㎓㎧₂➟⓫´❋㎠✳⑬⏐❻ＸⅩ⟹ⓐⓑ⧬＋❙．Ž├÷█☏�▼·㉢㎘󰊹⇢∆（


In [ ]:
symbol_final = symbol.translate(str.maketrans('','', '-―‌∙“”·‘’․~‑@%/#{}:;*.∼~～=&×㎢㎒㎎㎜㎍‰㎏㎓㎧㈔μ㏊' + string.punctuation + answer_except))

In [ ]:
print(symbol + '\n')
print(symbol_final)

.*○-□)ㅇ,❍※◦·「(ⅡⅠ><․▢Ⅲ!②」:①”?“‘]③Ⅳ%/’】【▪[~『‣④•‧◇■⇒▸』◈Ⅴ●－∙▶❚｢☞⑤→ㆍ❏◯㎡〇△《=･＊⇨▲&❑》㈜Ⅵ☐⑥➊'❖➋­◆∘+｣¦▣▷〕"⑦➌～❶❷×〔〈◉∼・〉↑◊⦁√⑧≪≫❸▫ᆞ➁➀–⋅➡_▹‑➂◌⁃Ⅶ➍￭❹►§▴❰❱➠—￮⃞⑨◾ｏ★◻㎥❒↓❐֠㎞੦➃▌#ⅱㆁ▮ⅰ➤✔：Ⅷ◎✤…‐󰊱⑩ⅲ〚➎☎▆⑴¡➄о⧭⑵󰊲✓$−@⧠➅㎢✍❺➭㎒♠㎎╺―‌１Ⅸ！⑪℃󰊳⌟±┗⁍➯↳➏⌜◀ʻ㎜ⅳ;Ÿ}⑶ᐅᄋ〛◐♣＃⋄󰋮➲➨✜▍㉰➫∎\㎝➐″⁎＜＞✽⧈∐➑㎍⚬{ⓛ┖ü⃝‥|㉮㉯㉱㉲㉳└‰〜⁜⑷㎖󰊴ⅵⅶ➒➓▯㎏⑫➛［］﹡➜㉠㉡⟶Œ）′​➆◤◢┎➝２㉴㉵➪⇑ㅣ❳￦➞⨯✲٠⁕㈔，┞⚫²ⅴ㏊μ㎓㎧₂➟⓫´❋㎠✳⑬⏐❻ＸⅩ⟹ⓐⓑ⧬＋❙．Ž├÷█☏�▼·㉢㎘󰊹⇢∆（

○□ㅇ❍※◦「ⅡⅠ▢Ⅲ②」①③Ⅳ】【▪『‣④•‧◇■⇒▸』◈Ⅴ●－▶❚｢☞⑤→ㆍ❏◯〇△《･＊⇨▲❑》Ⅵ☐⑥➊❖➋­◆∘｣¦▣▷〕⑦➌❶❷〔〈◉・〉↑◊⦁√⑧≪≫❸▫ᆞ➁➀–⋅➡▹➂◌⁃Ⅶ➍￭❹►§▴❰❱➠—￮⃞⑨◾ｏ★◻❒↓❐֠੦➃▌ⅱㆁ▮ⅰ➤✔：Ⅷ◎✤…‐󰊱⑩ⅲ〚➎☎▆⑴¡➄о⧭⑵󰊲✓−⧠➅✍❺➭♠╺１Ⅸ！⑪󰊳⌟┗⁍➯↳➏⌜◀ʻⅳŸ⑶ᐅᄋ〛◐♣＃⋄󰋮➲➨✜▍㉰➫∎➐″⁎＜＞✽⧈∐➑⚬ⓛ┖ü⃝‥㉮㉯㉱㉲㉳└⁜⑷㎖󰊴ⅵⅶ➒➓▯⑫➛［］﹡➜㉠㉡⟶Œ）′​➆◤◢┎➝２㉴㉵➪⇑ㅣ❳￦➞⨯✲٠⁕，┞⚫²ⅴ₂➟⓫´❋㎠✳⑬⏐❻ＸⅩ⟹ⓐⓑ⧬＋❙．Ž├÷█☏�▼·㉢󰊹⇢∆（


In [ ]:
def delete_symbols(con, mode='context'):
  con_ = ""
  for x in con.split('\n'):
    # 특수문자 제거
    x = x.translate(str.maketrans('','', symbol_final))
    
    # '-' 가장 첫 번째, 마지막 단어 제거
    if x.split()[0] == '-': # 첫 번째 단어 - 일 때, 첫 단어 '-'만 제거
      idx = x.find('-')
      x = x[:idx] + x[idx+1:] # 마지막 단어 '-'만 제거
      if x.split()[-1] == '-':
        idx = x.rfind('-')
        x = x[:idx] + x[idx+1:]

    # 개행 '. ' 으로 대체  
    if mode=='answer':
      return x
    con_ += x + '. ' 
  return con_

In [ ]:
print(train_df['context'][0])
print("============")
print(delete_symbols(train_df['context'][0]))

노후 어린이집 누수 제로 추진 결과 보고
Ⅰ 추진 근거
「영유아보육법 시행령」제12조
「서울특별시 강남구 영유아 보육 및 지원에 관한 조례」제15조, 제27조
2021년도 구립어린이집 확충 및 시설관리 계획 ( 2021.1.12.보육지원과-729호 ) 
Ⅱ 추진 개요
사업명 : 노후 어린이집 누수 긴급 개보수 공사
사업기간 : 2021. 4. 7. ~ 6. 25.
총사업비 : 금55,570,000원
사업대상 : 신사어린이집 외 3개소
노후 어린이집 누수 제로 추진 결과 보고.  추진 근거. 영유아보육법 시행령제12조. 서울특별시 강남구 영유아 보육 및 지원에 관한 조례제15조, 제27조. 2021년도 구립어린이집 확충 및 시설관리 계획 ( 2021.1.12.보육지원과-729호 ) .  추진 개요. 사업명 : 노후 어린이집 누수 긴급 개보수 공사. 사업기간 : 2021. 4. 7. ~ 6. 25.. 총사업비 : 금55,570,000원. 사업대상 : 신사어린이집 외 3개소. 


3) answer 처리

In [ ]:
train_df['answers'][0]

"{'text': ['금55,570,000원'], 'answer_start': [212]}"

In [ ]:
ans = eval(train_df['answers'][0])
answer_text = delete_symbols(ans['text'][0], mode='answer')
answer_text

'금55,570,000원'

In [ ]:
a = delete_symbols(train_df['context'][0])

In [ ]:
a[a.find(answer_text)]

'금'

4) 최종

In [ ]:
train_df = pd.read_csv("data/df_train_10_pre11.csv")
dev_df = pd.read_csv("data/df_dev_10_pre11.csv")

In [ ]:
def delete_symbols(con, mode='context'):
  con_ = ""
  for x in con.split('\n'):
    # 특수문자 제거
    x = x.translate(str.maketrans('','', symbol_final))
    
    # '-' 가장 첫 번째, 마지막 단어 제거
    if len(x.split()) > 0:
      if x.split()[0] == '-': # 첫 번째 단어 - 일 때, 첫 단어 '-'만 제거
        idx = x.find('-')
        x = x[:idx] + x[idx+1:] # 마지막 단어 '-'만 제거
        if len(x.split())>0 and x.split()[-1] == '-':
          idx = x.rfind('-')
          x = x[:idx] + x[idx+1:]

    # 개행 '. ' 으로 대체  
    if mode=='answer':
      return x
    con_ += x + '. ' 
  return con_

In [ ]:
# train_df = train_df['context'].apply(delete_symbols)

for i, x in enumerate(train_df['context']):
  train_df.loc[i, 'context'] = delete_symbols(x)

for i, x in enumerate(train_df['answers']):
  x = eval(x)
  if len(x['text']) > 0:
    answer_text = delete_symbols(x['text'][0], mode='answer')
    index = train_df['context'][i].find(answer_text)
    train_df.loc[i, 'answers'] = str({'text': [answer_text], 'answer_start': [index]})

for i, x in enumerate(dev_df['context']):
  dev_df.loc[i, 'context'] = delete_symbols(x)

for i, x in enumerate(dev_df['answers']):
  x = eval(x)
  if len(x['text']) > 0:
    answer_text = delete_symbols(x['text'][0], mode='answer')
    index = dev_df['context'][i].find(answer_text)
    dev_df.loc[i, 'answers'] = str({'text': [answer_text], 'answer_start': [index]})

In [ ]:
train_df.to_csv('data/df_train_10_prep2.csv')
dev_df.to_csv('data/df_dev_10_prep2.csv')

In [ ]:
a = "2021년 주민참여예산 운영계획 ( 안 ) .  관련근거.  행정안전부지방공공기관 일하는 방식 혁신방안 통보 ( 2018.6.19. ) .  서울특별시지방공기업 주민 참여소통 확대방안 시행 요청 ( 2018.8.10. ) .  행정안전부2020년 지방공공기관 종합 정책방향수립 알림 ( 2020.4.1. ) .  추진배경 및 현황.  추진배경.  정부․시의 ‘예산편성 시 주민이 참여하는 제도’ 도입 요구.  지역주민들과 소통하는 기업의 사회적 책임 실현.  주민참여 예산 ( 안 ) .  추진방향.  주민참여 확대를 통한 예산편성의 객관성, 공정성 제고.  주민의 관점에서 필요한 사업비 직접 제안 및 사업선정 직접 참여.  주민과 정보를 공유함으로써 공사 재정운영에 대한 관심 유도.  주민의견 수렴 결과 공개를 통한 예산편성의 투명성 제고.  추진개요.  추진기간 : 2021. 6월~12월.  예산규모 : 공사 재정현황 감안 예산 범위 내.  사업대상 : 인건비․법적 필수경비 포함사업을 제외한 주민 밀착 공익 사업.  주민참여예산 제안사업 접수.  접수기간 : 2020.8.1.~2021.7.31..  사업범위 : 서울지하철1~8호선 운영에 관한 업무 ( 타기관관할구간제안사업검토제외 ) .  제안대상 : 서울지하철을 이용하며 느낀 불편사항을 개선하거나, 주민 편익 증진을 위하여 필요한 사업.  접수방법 : 인터넷 공사 홈페이지를 통한 접수.  제안사업 관련 해당부서 검토의견 수렴 : 2021.8.3.~8.13.. "

In [ ]:
a[573]

'서'